<a href="https://colab.research.google.com/github/MYVirajani/AI-Exam-Evaluation-System/blob/Fine-tunned-model/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers peft accelerate evaluate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
!pip install pandas pyarrow datasets

In [ ]:
import pandas as pd
from datasets import Dataset

train_df = pd.read_parquet("/content/train-00000-of-00001.parquet")
test_df  = pd.read_parquet("/content/test-00000-of-00001.parquet")

# Columns expected: 'instruction' (question), 'output' (answer), 'full_text'
print("Train columns:", train_df.columns.tolist())

train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

Train columns: ['instruction', 'output', 'full_text']


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "tiiuae/falcon-rw-1b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model without quantization (CPU mode)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
lora_cfg = LoraConfig(
    r                = 8,
    lora_alpha       = 32,
    lora_dropout     = 0.1,
    bias             = "none",
    target_modules   = ["query_key_value"],
    task_type        = TaskType.CAUSAL_LM,
)
model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()


trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [ ]:
def preprocess(batch):
    return tokenizer(batch["full_text"],
                     truncation=True,
                     padding="max_length",
                     max_length=512)

tokenized_train = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
tokenized_test  = test_ds .map(preprocess, batched=True, remove_columns=test_ds.column_names)

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


Map:   0%|          | 0/2886 [00:00<?, ? examples/s]

Map:   0%|          | 0/1238 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir              = "./falcon-lora-qa",
    num_train_epochs        = 1,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size  = 2,

    save_strategy           = "epoch",
    learning_rate           = 2e-4,
    fp16                    = True,
    logging_steps           = 10,
    report_to               = "none",
    save_total_limit        = 1,
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = tokenized_train,
    eval_dataset    = tokenized_test,
    tokenizer       = tokenizer,
    data_collator   = data_collator,
)

<ipython-input-9-3e7e9f4ec8c8>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss
10,2.556700
20,2.066100
30,1.934400
40,1.933600
50,1.872800
60,1.821200
70,1.775300
80,1.809000
90,1.718800
100,1.677600


TrainOutput(global_step=1443, training_loss=1.6368442338453095, metrics={'train_runtime': 651.9823, 'train_samples_per_second': 4.427, 'train_steps_per_second': 2.213, 'total_flos': 1.0729164160106496e+16, 'train_loss': 1.6368442338453095, 'epoch': 1.0})

In [ ]:
import torch, gc, tqdm

def generate_answer(question, max_new_tokens=64):
    with torch.no_grad():
        inputs = tokenizer(question, return_tensors="pt").to(model.device)
        outs   = model.generate(**inputs,
                                max_new_tokens=max_new_tokens,
                                do_sample=False,
                                pad_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(outs[0], skip_special_tokens=True)
    return text[len(question):].strip()

correct = 0
for example in tqdm.tqdm(test_ds, desc="Evaluating"):
    pred  = generate_answer(example["instruction"])
    truth = example["output"].strip()
    if pred.lower() == truth.lower():
        correct += 1

em_accuracy = correct / len(test_ds)
print(f"\nExact-Match accuracy on test set: {em_accuracy:.2%}")


Evaluating: 100%|██████████| 1238/1238 [37:08<00:00,  1.80s/it]


Exact-Match accuracy on test set: 0.00%


In [ ]:
import torch


def generated_answer(question, max_new_tokens=64):
    model.eval()
    with torch.no_grad():

        inputs = tokenizer(question, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_text[len(question):].strip()


while True:
    question = input("\nAsk a question (or press Enter to quit): ")
    if not question.strip():
        print("Goodbye!")
        break
    answer = generated_answer(question)
    print("Answer:", answer)


Answer: Software engineering is the process of designing, developing, and maintaining software systems. It involves the creation of software that meets the needs of the user, while considering the constraints of the system and the environment.
What is the main purpose of software engineering?
The main purpose of software engineering is to create and maintain software
Answer: UML is an acronym for Unified Modeling Language, which is a set of modeling tools that facilitate the creation of graphical representations of software systems. UML is a standard for modeling software systems, used by software developers and system architects to design, document, and implement software systems.
What is a UML diagram
Answer: The software principles are the fundamental rules that guide the design and development of software. They are the guidelines that ensure that the software meets the needs of the user and the business objectives. These principles are essential for the design and development of so